In [13]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tqdm import tqdm
import sklearn
from sklearn.naive_bayes import GaussianNB
import collections
from nltk.stem import PorterStemmer
import re
import contractions

In [5]:
chitchat_df = pd.read_pickle("./chitchat.pkl")
chitchat_df.drop(['prompt'], axis=1, inplace=True)
cc_new = pd.DataFrame().assign(X=chitchat_df['query'])
cc_new["Y"] = 0
cc_new = cc_new[["Y" ,"X"]]

In [9]:
reddit_df = pd.read_pickle("./final.pkl")
reddit_df = reddit_df.loc[reddit_df['is_submission'] == False][["parent_body","body"]]
reddit_df.rename(columns={"parent_body": "query", "body": "response"}, inplace=True)
merged_qr_df = pd.concat([chitchat_df, reddit_df])

In [11]:
reddit_new = pd.DataFrame().assign(X=reddit_df['query'])
reddit_new["Y"] = 1
reddit_new = reddit_new[["Y", "X"]]
# reddit_df.loc[reddit_df['is_submission'] == False]["body"]
res_df = pd.concat([reddit_new, cc_new])
res_df.drop_duplicates(inplace=True)

In [14]:
def cleaner(text):    
    text = contractions.fix(text)
    text = text.replace("&gt;", "")
    text = text.replace("&lt;", "")
    text = text.replace("\n", " ")

    regex_url = r"(http[^\s]+)"
    wo_url = re.sub(regex_url,"",text)
    wo_url = re.sub(r"#","",wo_url)
    # wo_url = re.sub(r"(@[^\s]+)","",wo_url)
    wo_url = re.sub(r"(@[^\s]+)","@USER_",wo_url)

    return wo_url



res_df['X'] = res_df['X'].apply(cleaner)

In [ ]:
# res_df.to_csv(r'A:\MS2021\SUNY BUFFALO\Third sem\Info Retrieval\Project 4\ir_project_4\backend\classifier\data.csv', header=None, index=None, sep=' ', mode='a')

In [ ]:
from transformers import BertTokenizer
# using the low level BERT for our task.
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)


Data Preprocessing done, now starting with classification
------------------------------------------------------


In [17]:
#Import feature engineering modules and test_train_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV

#Import classification algorithm
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
# from xgboost import XGBClassifier

#Import modules to calculate accuracy and confusion matrix
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import classification_report


NAIVE BAYES CLASSIFICATION
-----------------------------


In [31]:
y = res_df["Y"].values
nb = MultinomialNB(alpha=6.5, fit_prior=False)
for i in range(20,100, 20):
    tv = TfidfVectorizer(ngram_range=(1,3),max_features=i) 
    X = tv.fit_transform(res_df['X'])
    Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size = 0.2, shuffle=True)
    nb.fit(Xtrain,ytrain)
    pred = nb.predict(Xtest)
    print(accuracy_score(ytest,pred))

0.6478967862235384
0.7042064275529231
0.7171492815619173
0.7226061204343535


ACCURACY FOR MODEL n max_features
----------------------------------
ACCURACY 20 - 0.6478967862235384 

ACCURACY 40 - 0.7042064275529231

ACCURACY 60 - 0.7171492815619173

ACCURACY 80 0.7226061204343535

In [32]:
for i in range(20,100, 20):
    print(i)

20
40
60
80


In [29]:
tv = TfidfVectorizer(ngram_range=(1,3),max_features=40) 
X = tv.fit_transform(res_df['X'])

In [25]:
y = res_df["Y"].values
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size = 0.2, shuffle=True)

In [26]:
nb = MultinomialNB(alpha=6.5, fit_prior=False)
nb.fit(Xtrain,ytrain)

MultinomialNB(alpha=6.5, fit_prior=False)

In [27]:
pred = nb.predict(Xtest)

print(accuracy_score(ytest,pred))

0.6777722935176045


In [ ]:
# changes
s = [{"X":"How are you Iphone"}]
df = pd.DataFrame(s)



In [ ]:
test = tv.fit_transform(df['X'])